In [1]:
import dataSrc
import utilities
import pandas as pd

In [2]:
# Retrieve symbols and financials

sp500_tickers = dataSrc.sp500_symbols()
dow_tickers = dataSrc.dow_symbols()

In [3]:
# sp500_financials = dataSrc.financials(sp500_tickers)

In [4]:
df = dataSrc.financials(dow_tickers)

CRM: 1/30
WBA: 2/30
V: 3/30
NKE: 4/30
UNH: 5/30
TRV: 6/30
VZ: 7/30
INTC: 8/30
WMT: 9/30
JNJ: 10/30
DIS: 11/30
MCD: 12/30
JPM: 13/30
CAT: 14/30
BA: 15/30
AMGN: 16/30
DOW: 17/30
AAPL: 18/30
GS: 19/30
CSCO: 20/30
MSFT: 21/30
HD: 22/30
PG: 23/30
MRK: 24/30
IBM: 25/30
HON: 26/30
KO: 27/30
CVX: 28/30
AXP: 29/30
MMM: 30/30


In [6]:
# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

In [7]:
conditions = (df['Revenue_Growth'] > 0) & \
            (df['GPMargin'] > 0)& \
            (df['EPS_Growth'] > 0)& \
            (df['ROE'] > 0) & \
            (df['DPS_Growth'] > 0) & \
            (df['DivYield'] > 0)
df = df[conditions]

In [9]:
# Compute historical momentum
# Average momentum of prev 6M, 12M and 24M

mom_list = []
for symbol in df['symbol']:
    print(symbol)
    m12_momentum = utilities.calculate_hist_momentum(symbol, 252)
    m24_momentum = utilities.calculate_hist_momentum(symbol, 504)
    m36_momentum = utilities.calculate_hist_momentum(symbol, 756)
    avg_momentum = (m12_momentum + m24_momentum + m36_momentum) / 3
    mom_list.append(avg_momentum)

df['momentum'] = mom_list
df

WBA
NKE
TRV
VZ
INTC
JNJ
CSCO
MSFT
HD
CVX


,symbol,name,exchange,sector,industry,marketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,momentum
1,WBA,"Walgreens Boots Alliance, Inc.",NASDAQ,Healthcare,Pharmaceutical Retailers,42.315162,0.441947,0.106518,0.202202,0.163854,0.038085,1.8715,0.304819,-0.042686
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,268.667912,0.595383,0.514070,0.448202,0.043478,0.006270,1.0700,0.716614,0.943619
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,39.584559,0.481540,0.132920,0.249376,0.280277,0.021586,3.4300,0.317382,0.226135
6,VZ,Verizon Communications Inc.,NYSE,Communication Services,Telecom Services,232.157233,0.089299,0.281949,0.586553,0.102362,0.044696,2.5110,0.110776,-0.009961
7,INTC,Intel Corporation,NASDAQ,Technology,Semiconductors,213.641609,0.743302,0.231499,0.555936,0.506024,0.026058,1.3730,0.374296,0.105223
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,470.397846,0.338716,0.270051,0.669963,0.012766,0.022789,4.0900,0.348304,0.302498
19,CSCO,"Cisco Systems, Inc.",NASDAQ,Technology,Communication Equipment,235.363074,0.272511,0.263007,0.639438,0.133333,0.026067,1.4600,0.422486,0.243103
20,MSFT,Microsoft Corporation,NASDAQ,Technology,Software—Infrastructure,2196.301939,1.328849,0.462317,0.689258,0.064902,0.009383,2.7500,0.553834,1.075348
21,HD,"The Home Depot, Inc.",NYSE,Consumer Cyclical,Home Improvement Retail,337.040671,0.925909,6.827419,0.339370,0.453451,0.019641,6.3000,1.407165,0.436488
27,CVX,Chevron Corporation,NYSE,Energy,Oil & Gas Integrated,195.847062,0.266313,0.026990,0.290348,1.236111,0.065025,6.5500,0.254691,-0.045244


In [17]:
df['mom_rank'] = df['momentum'].rank()
df = df.sort_values(by=['mom_rank'], ascending=False)
# numRows = df.shape[0]
# numCols = df.shape[1]
# top10 = df.copy()
df = df[df['momentum'] > 0]

In [18]:
watchlist = df[['symbol','name']].copy()
watchlist

,symbol,name
20,MSFT,Microsoft Corporation
3,NKE,"NIKE, Inc."
21,HD,"The Home Depot, Inc."
9,JNJ,Johnson & Johnson
19,CSCO,"Cisco Systems, Inc."
5,TRV,"The Travelers Companies, Inc."
7,INTC,Intel Corporation


In [19]:
# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []

for symbol in watchlist['symbol']:
    print(symbol)
    currentPrice = utilities.get_current_price(symbol)
    high = utilities.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high
    
    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)
    

watchlist['CurrentPrice'] = currentPrices
watchlist['52W_High'] = highs
watchlist['Discount%'] = discounts
    

MSFT
NKE
HD
JNJ
CSCO
TRV
INTC


In [20]:
watchlist = watchlist.sort_values(by='Discount%')

In [21]:
watchlist

,symbol,name,CurrentPrice,52W_High,Discount%
7,INTC,Intel Corporation,52.69,68.489998,-0.230691
21,HD,"The Home Depot, Inc.",320.75,345.690002,-0.072146
5,TRV,"The Travelers Companies, Inc.",158.90,162.710007,-0.023416
3,NKE,"NIKE, Inc.",170.66,174.380000,-0.021333
19,CSCO,"Cisco Systems, Inc.",56.01,56.620000,-0.010774
20,MSFT,Microsoft Corporation,293.08,294.820000,-0.005902
9,JNJ,Johnson & Johnson,179.47,177.890000,0.008882
